<a href="https://colab.research.google.com/github/Miokasa/Odeuropa/blob/main/Odeuropa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#!unzip /content/drive/MyDrive/Odeuropa/webanno.zip -d /content/drive/MyDrive/Odeuropa/webanno

Creation of folds for train/dev/test

In [15]:
!python3 /content/drive/MyDrive/Odeuropa/create_folds.py \
  --folder /content/drive/MyDrive/Odeuropa/webanno \
  --output /content/drive/MyDrive/Odeuropa/folds_output \
  --tasktype BERT \
  --tags Smell\\_Word,Smell\\_Source,Quality

 tag: {'Circumstances': 0, 'Effect': 1, 'Evoked\\_Odorant': 2, 'Location': 3, 'Odour\\_Carrier': 4, 'Perceiver': 5, 'Quality': 6, 'Smell\\_Source': 7, 'Smell\\_Word': 8, 'Time': 9}


Convert English plain texts to Bert format for the classification

In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

csv_path = "/content/drive/MyDrive/Odeuropa/DocumentsEN.csv"
save_path = "/content/drive/MyDrive/Odeuropa/books_raw"
os.makedirs(save_path, exist_ok=True)


df = pd.read_csv(csv_path)

for idx, row in df.iterrows():
    doc_id = str(row['Document Identifier']) + ".txt"
    url = row['Link to source']

    try:
        response = requests.get(url, timeout=10)
        response.encoding = response.apparent_encoding
        if url.endswith(".txt"):
            text = response.text
        else:
            soup = BeautifulSoup(response.text, "html.parser")

            if "wikisource" in url:
                main_text = soup.find("div", {"class": "mw-parser-output"})
            elif "umich.edu" in url:
                main_text = soup.find("div", {"class": "text"})
            else:
                main_text = soup.get_text()

            if main_text:
                text = main_text.get_text(separator="\n")
            else:
                text = soup.get_text()

        with open(os.path.join(save_path, doc_id), "w", encoding="utf-8") as f:
            f.write(text)

        print(f"Saved {doc_id}")

    except Exception as e:
        print(f"Failed to fetch {doc_id} from {url}: {e}")


Saved 001E.txt
Saved 002E.txt
Saved 003E.txt
Saved 004E.txt
Saved 005E.txt
Failed to fetch 006E.txt from https://wellcomelibrary.org/moh/report/b19874455/8#?c=0&m=0&s=0&cv=0: 'str' object has no attribute 'get_text'
Failed to fetch 007E.txt from https://www.gutenberg.org/files/60584/60584-h/60584-h.htm: 'str' object has no attribute 'get_text'
Saved 008E.txt
Saved 009E.txt
Failed to fetch 010E.txt from https://www.gutenberg.org/files/829/829-h/829-h.htm: 'str' object has no attribute 'get_text'
Saved 011E.txt
Saved 012E.txt
Saved 013E.txt
Saved 014E.txt
Saved 015E.txt
Saved 016E.txt
Failed to fetch 017E.txt from https://archive.org/details/b20398177: 'str' object has no attribute 'get_text'
Failed to fetch 018E.txt from https://archive.org/details/b21481246: 'str' object has no attribute 'get_text'
Failed to fetch 019E.txt from https://archive.org/details/b2041075x: 'str' object has no attribute 'get_text'
Failed to fetch 020E.txt from https://archive.org/details/b28116598/page/n1/mode

In [17]:
!python3 /content/drive/MyDrive/Odeuropa/books_converter.py \
  --folder /content/drive/MyDrive/Odeuropa/books_raw \
  --output /content/drive/MyDrive/Odeuropa/converted_books\
  --label EN \
  --books 1

Odeuropa Smell Classifier

In [18]:
#!unzip /content/drive/MyDrive/Odeuropa/bert-base-uncased-odeuropa-20250607T180019Z-1-001.zip -d /content/drive/MyDrive/Odeuropa/model/

In [19]:
!pip install torch==2.0.1 tqdm==4.63.0 transformers==4.17.0 seqeval[gpu]==1.2.2 sentencepiece==0.1.96 datasets==2.0.0 optuna==2.10.0

In [23]:
!python3 /content/drive/MyDrive/Odeuropa/smell_classifier.py \
  --model_path "/content/drive/MyDrive/Odeuropa/model/bert-base-uncased-odeuropa" \
  --input_path "/content/drive/MyDrive/Odeuropa/converted_books" \
  --output_path "/content/drive/MyDrive/Odeuropa/predictions" \
  --lang "english"

2025-06-20 16:37:52.256095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750437472.661849    8684 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750437472.772149    8684 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-20 16:37:53.625599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
version

In [27]:
!python3 /content/drive/MyDrive/Odeuropa/extract_annotations.py \
  --folder "/content/drive/MyDrive/Odeuropa/predictions" \
  --smellwordtag Smell_Word \
  --tags Smell_Source,Quality \
  --stopwords "/content/drive/MyDrive/Odeuropa/stopwords.txt" \
  --output "/content/drive/MyDrive/Odeuropa/extracted_frame_elements.tsv"
